# Storytelling dei flussi

In [1]:
import pandas as pd
import numpy as np



/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_88081/2504811429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Creazione del Task da dati storici

In [2]:
from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master

from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

In [3]:
# help(Task)

In [4]:
# Instanziare un oggetto task

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= 300,
            robustness_risk= 0.2,
            robustness_overtime= 60, # TODO check robustness_overtime unit of measure
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
)

In [5]:
# Importare i dati e istanziare un oggetto patient provider

historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [6]:
historical_data_df.urgency

0       3
1       1
2       0
3       3
4       0
       ..
9155    1
9156    4
9157    1
9158    1
9159    4
Name: urgency, Length: 9160, dtype: int64

In [7]:
# Campionatura di singoli pazienti
patient = patient_provider.provide_patient()
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='C', requested_urgency=3)
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='B', requested_urgency=3)
print(patient)

Patient id: 5298 
 equipe: G 
 urgency: 4
Patient id: 7481 
 equipe: C 
 urgency: 3
Patient id: 8714 
 equipe: B 
 urgency: 3


In [8]:
# Inserire un insieme di pazienti in task
task.patients = patient_provider.provide_patient_set(quantity=300)

## Previsione del tempo operatorio

In [9]:
from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal

In [10]:
patients, training = patient_provider.provide_sets(quantity=10, quantity_training=100)


In [11]:
patients

In [12]:
patients[0].features

array([3, 0, 0, ..., 0.0, 0.0, 0.0], dtype=object)

In [13]:
# instantiating the model, and training

model = NGBNormal(description="test", patients=training)

In [14]:
patients_prediction = model.predict(patients)

In [15]:
patients_prediction[0].uncertainty_profile.param_loc

128.74437012654414

In [16]:
patients_prediction[0].uncertainty_profile.param_scale

24.103471444246846

In [17]:
patients_prediction[0].uncertainty_profile.sample(10)

array([122.19756244, 127.96037846, 131.90363054, 102.93238268,
       115.85312874, 128.71733197, 118.32048576, 176.51521727,
       123.16312098, 127.68478018])

## Creazione del Task con pazienti completi

In [18]:
patients, training = patient_provider.provide_sets(quantity=300, quantity_training=300)


In [19]:
model = NGBLogNormal(description="test", patients=training)

patients = model.predict(patients)

In [20]:

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= 300,
            robustness_risk= 0.2,
            robustness_overtime= 0.9,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients


In [21]:
patients = task.patients
patients[1].max_waiting_days

7

In [22]:
patients[1].days_waiting

3

In [23]:
print(patients[0])

Patient id: 4111 
 equipe: L 
 urgency: 0 
 nominal duration: 343.2773687697954


## Creazione del task con il Master Scheduling

In [24]:
or_master_schedule = pd.read_csv("../data/master_schedule_input.csv", sep = ';')
or_master_schedule

,weekday,equipes,room,duration
0,1,"C, D",or1,300
1,1,"D, C",or1,300
2,1,I,or2,600
3,2,"H, F",or1,300
4,2,"D, C",or1,300
5,2,L,or2,600
6,3,"C, D",or1,300
7,3,"F, H",or1,300
8,3,I,or1,300
9,3,"H, F",or2,600


In [25]:
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [26]:
print(master_schedule)

Master 'my master' - 16 blocks on 5 days.


In [27]:
task.master_schedule = master_schedule

In [28]:
for block in master_schedule.blocks:
    print(block)
    print('----\n')
    
    

Master Block number 0
                
Equipe: ['C', 'D']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 1
                
----

Master Block number 1
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 2
                
----

Master Block number 2
                
Equipe: ['I']
                
Duration: 600
                
Room: or2
                
Weekday: 1
                
Number in day: 3
                
----

Master Block number 3
                
Equipe: ['H', 'F']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 1
                
----

Master Block number 4
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 2
                
----



## Creazione dell'ottimizzatore

In [29]:
implementor_adversary = ImplementorAdversary(task = task, implementor=StandardImplementor(), adversary=None, description = 'Test')



In [30]:
implementor_adversary._task



In [31]:
implementor_adversary.create_instance()



In [32]:
implementor_adversary._instance.get(None).get('n_pats')

{None: 300}

In [33]:
hello = implementor_adversary.run_implementor()


In [ ]:
for pat in (hello._blocks[10].patients):
    print(pat)

In [ ]:
hello._blocks[4].patients[0].uncertainty_profile.param_s

0.2963662971803107

In [ ]:
task.master_schedule.blocks

In [ ]:
task.patients

In [ ]:
task.patients[0].uncertainty_profile

## Creazione dell'ottimizzatore

In [ ]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor

In [ ]:
implementor_adversary = VanillaImplementor(task = task, implementor=ChanceConstraintsImplementor(task = task), description = 'Test')



In [ ]:
implementor_adversary._task



In [ ]:
implementor_adversary.create_instance()



In [ ]:
implementor_adversary._instance.get(None).get('n_pats')

{None: 300}

In [ ]:
hello = implementor_adversary.run_implementor()


RuntimeError: A feasible solution was not found, so no solution can be loaded.Please set opt.config.load_solution=False and check results.termination_condition and results.best_feasible_objective before loading a solution.

In [ ]:
implementor_adversary._instance.get(None).get('f')

In [ ]:
for pat in (hello._blocks[0].patients):
    print(pat)

In [ ]:
hello._blocks[0].patients[0].uncertainty_profile

In [ ]:
hello.x[1,1]()

In [ ]:
task.master_schedule.blocks

In [ ]:
task.patients

In [ ]:
task.patients[0].uncertainty_profile